# Notebook 5: Composing Examples

In this notebook we will examine how we can use the elements we have encountered so far, in order to construct examples which will allow us to train machine learning models with data generated in real time. This is a core functionality of GravyFlow.

As usual, we begin by performing the relevent imports:

In [1]:
import os
os.environ['KERAS_BACKEND'] = 'jax'

# Built-in imports
from typing import List, Dict, Iterator
from pathlib import Path

# Dependency imports: 
import numpy as np
import keras
from keras import ops
import jax
import jax.numpy as jnp
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

# Import the GravyFlow module.
import gravyflow as gf

INFO:2025-12-03 14:39:07,895:jax._src.xla_bridge:812: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
2025-12-03 14:39:07,895 - INFO - Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
W1203 14:39:09.885478 3644687 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1203 14:39:09.885966 3644689 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1203 14:39:09.886253 3644688 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1203 14:39:09.886496 3644684 cuda_executor.cc:1802] GPU interconne

## Creating an example generator through composition

We can combine all the elements we have seen so, noise generation, waveform generation, and waveform projection, and use MLy to create a custom Python generator.

In order to scale the injection with respect to the noise we can use a `gf.ScalingMethod` object. GravyFlow supports scaling with SNR (`gf.ScalingTypes.SNR`), HRSS (`gf.ScalingTypes.HRSS`), and HPEAK (`gf.ScalingTypes.HPEAK`).

- `value` : `Union[gf.Distribution, np.ndarray]`, Required
  > The value or distribution to use to scale the injections, units vary depending on type parameter.

- `type_` : `gf.ScalingTypes`, Required
  > Type of scaling, one of either `gf.ScalingTypes.SNR`, `gf.ScalingTypes.HRSS`, or `gf.ScalingTypes.HPEAK`.

Let us create a `gf.ScalingMethod` object:

In [2]:
# Create a scaling method object in order to scale the injection to the noise:
scaling_method : gf.ScalingMethod = gf.ScalingMethod(
    value=gf.Distribution(
        value=20,
        type_=gf.DistributionType.CONSTANT
    ),
    type_=gf.ScalingTypes.SNR
)

Next, we can set up all the other elements that we can use to compose or example generator:

In [ ]:
# This object will be used to obtain real interferometer data based on specified parameters.
ifo_data_obtainer : gf.IFODataObtainer = gf.IFODataObtainer(
    observing_runs=gf.ObservingRun.O3, # Specify the observing run (e.g., O3).
    data_quality=gf.DataQuality.BEST,  # Choose the quality of the data (e.g., BEST).
    data_labels=[                      # Define the types of data to include.
        gf.DataLabel.NOISE, 
        gf.DataLabel.GLITCHES
    ],
    segment_order=gf.SegmentOrder.RANDOM, # Order of segment retrieval (e.g., RANDOM).
    force_acquisition=True,               # Force the acquisition of new data.
    cache_segments=False                  # Choose not to cache the segments.
)

# Initialize the noise generator wrapper:
# This wrapper will use the ifo_data_obtainer to generate real noise based on the specified parameters.
noise: gf.NoiseObtainer = gf.NoiseObtainer(
    ifo_data_obtainer=ifo_data_obtainer, # Use the previously set up IFODataObtainer object.
    noise_type=gf.NoiseType.WHITE,        # Specify the type of noise as REAL.
    ifos=[gf.IFO.L1, gf.IFO.H1] # Specify the interferometer (e.g., LIGO Livingston L1).
)

# Define a uniform distribution for the mass of the first object in solar masses.
mass_1_distribution_msun : gf.Distribution = gf.Distribution(
    min_=10.0, 
    max_=60.0, 
    type_=gf.DistributionType.UNIFORM
)

# Define a uniform distribution for the mass of the second object in solar masses.
mass_2_distribution_msun : gf.Distribution = gf.Distribution(
    min_=10.0, 
    max_=60.0, 
    type_=gf.DistributionType.UNIFORM
)

# Define a uniform distribution for the inclination of the binary system in radians.
inclination_distribution_radians : gf.Distribution = gf.Distribution(
    min_=0.0, 
    max_=np.pi, 
    type_=gf.DistributionType.UNIFORM
)
    
# Initialize a RippleGenerator with the defined distributions and specific approximant.
# Available approximants: "IMRPhenomD", "IMRPhenomXAS", "IMRPhenomPv2", "TaylorF2", "IMRPhenomD_NRTidalv2"
ripple_generator : gf.WaveformGenerator = gf.RippleGenerator(
    approximant="IMRPhenomPv2", # Specify the waveform approximant here
    mass_1_msun=mass_1_distribution_msun,
    mass_2_msun=mass_2_distribution_msun,
    inclination_radians=inclination_distribution_radians,
    scaling_method=scaling_method,
    injection_chance=0.5,
    # For IMRPhenomPv2, you might want to specify spins if not using defaults:
    # spin_1_in=(0.0, 0.0, 0.0), 
    # spin_2_in=(0.0, 0.0, 0.0)
)
    
generator : Iterator = gf.data(       
    noise_obtainer=noise,
    waveform_generators=ripple_generator, # Use the ripple generator
    num_examples_per_batch=8,
    input_variables=[
        gf.ReturnVariables.WHITENED_ONSOURCE, 
    ],
    output_variables=[
        gf.ReturnVariables.INJECTIONS, 
        gf.ReturnVariables.WHITENED_INJECTIONS,
        gf.ReturnVariables.INJECTION_MASKS
    ]
)

/home/michael.norman/gravyflow/gravyflow/src/dataset/dataset.py:31: UserWarning: Noise is not REAL or PSEUDO-REAL, yet data obtainer is defined.
  warn(
/home/michael.norman/gravyflow/gravyflow/src/dataset/dataset.py:49: UserWarning: Whitening requested for WHITE NOISE.
  warn(


Similiarly to the individual elements, we can use this example generator as an iterator, and produce N examples for use to examine:

In [4]:
# Generate a batch of examples using the composed generator.
input_data, output_data = next(generator)

We can print the output of the generator to examine its format, which values are returned will depend on which parameters we have requested in the input_variables, and output_variables field in our gf.data function. Both are returned in the form of a dictionary, which can easly be fed into a keras model if the inputs of the model are named similarly to the variable fields, we will show an example of this later in the notebook.

In [5]:
# This is the data we will uses as an input examples to out model:
print(f"Dictionary to feed the model: \n {input_data}")

# This is the target data we will use as labels to train our model:
print(f"Dictionary to use as the model labels: \n {output_data}")

Dictionary to feed the model: 
 {'WHITENED_ONSOURCE': Array([[[-0.04132,  1.167  ,  1.071  , ...,  0.4675 ,
          0.5244 ,  1.164  ],
        [ 0.1007 , -0.512  , -0.914  , ...,  0.1973 ,
          0.9487 , -0.2219 ]],

       [[ 1.708  ,  1.275  ,  0.982  , ..., -1.68   ,
          0.2465 , -1.5205 ],
        [-0.816  ,  1.018  , -0.4148 , ..., -0.4597 ,
          1.707  , -0.936  ]],

       [[ 0.2756 , -1.931  ,  2.152  , ..., -0.9995 ,
          0.083  ,  1.179  ],
        [-0.8027 ,  0.39   ,  0.4587 , ..., -1.135  ,
         -0.3838 , -0.603  ]],

       ...,

       [[-1.091  , -1.517  , -0.09717, ...,  0.03014,
         -2.408  , -0.5796 ],
        [ 2.     , -0.451  , -3.11   , ...,  0.0811 ,
         -0.5107 ,  0.562  ]],

       [[-0.674  ,  0.1467 , -0.7954 , ...,  1.739  ,
          0.888  ,  0.919  ],
        [ 0.8286 , -1.494  ,  0.918  , ...,  0.5713 ,
         -0.524  , -0.01478]],

       [[ 0.2036 , -0.238  ,  2.213  , ..., -1.324  ,
         -0.3506 , -0.674  ],

We can then print some examples from this dataset to examine the output:

In [6]:
# Extract the data from the generator output: 
onsource: jax.Array = input_data[gf.ReturnVariables.WHITENED_ONSOURCE.name]
injections: jax.Array = output_data[gf.ReturnVariables.INJECTIONS.name][0]
whitened_injections: jax.Array = output_data[gf.ReturnVariables.WHITENED_INJECTIONS.name][0]
injection_masks: jax.Array = output_data[gf.ReturnVariables.INJECTION_MASKS.name][0]

# Initialize an empty layout for the strain plots.
generated_data_layout: List = []

# Iterate over the multi-injections and their corresponding parameters.
for onsource_, injection, whitened_injection, masks_ in zip(
        onsource,
        injections,
        whitened_injections,
        injection_masks
    ):
    # Create strain plots for each Phenom D and WNB injection with titles displaying the parameter values.
    generated_data_layout.append([
        gf.generate_strain_plot(
            {
                "Onsouce": onsource_,
                "Whitened Injection": whitened_injection,
                "Injection": injection,
            },
            height=400,
            width=800,
            title=f"Example Output. Has Injection: {bool(masks_)}"
        )
    ])

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(generated_data_layout)
output_notebook()
show(grid)

Loading BokehJS ...